In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CBS(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(CBS, self).__init__()
        # Convolutional layer
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # Batch Normalization
        self.bn = nn.BatchNorm2d(out_channels)
        # SiLU Activation function
        self.silu = nn.SiLU()

    def forward(self, x):
        # Forward pass through the CBS block
        x = self.conv(x)
        x = self.bn(x)
        x = self.silu(x)
        return x

In [ ]:
class SPPBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SPPBlock, self).__init__()
        
        # Initial convolution to reduce channels before pooling
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
        # The pooling layers with different kernel sizes
        self.pool1 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=9, stride=1, padding=4)
        self.pool3 = nn.MaxPool2d(kernel_size=13, stride=1, padding=6)
        
        # Final convolution to combine the pooled features
        self.conv2 = nn.Conv2d(out_channels * 4, out_channels, kernel_size=1)
        
    def forward(self, x):
        # Initial convolution
        x = self.conv1(x)
        
        # Apply the different pooling operations at different scales
        pool1_out = self.pool1(x)
        pool2_out = self.pool2(x)
        pool3_out = self.pool3(x)
        
        # Concatenate the pooled outputs along the channel dimension
        x = torch.cat([x, pool1_out, pool2_out, pool3_out], dim=1)
        
        # Apply final convolution to reduce the channels
        x = self.conv2(x)
        
        return x

In [ ]:
class C3Block(nn.Module):
    def __init__(self, in_channels, out_channels, num_conv=1, groups=1):
        super(C3Block, self).__init__()

        # Initial Conv Layer (1x1)
        self.conv1 = nn.Conv2d(in_channels, out_channels // 2, kernel_size=1)

        # Sequential 3x3 Convolutions
        self.convs = nn.ModuleList([nn.Conv2d(out_channels // 2, out_channels // 2, kernel_size=3, padding=1) for _ in range(num_conv)])

        # Final 1x1 convolution to adjust the output channels
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1)

    def forward(self, x):
        # First part: Apply 1x1 convolution
        x1 = self.conv1(x)

        # Second part: Apply multiple 3x3 convolutions (num_conv layers)
        x2 = x1
        for conv in self.convs:
            x2 = conv(x2)

        # Concatenate the outputs from both paths
        x = torch.cat([x1, x2], dim=1)

        # Final 1x1 convolution to reduce the concatenated channels
        x = self.conv2(x)

        return x